Predicting the wine quality using word vectors

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
df=pd.read_csv("winemag-data-130k-v2.csv",index_col=0)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
df['country'].unique()

array(['Italy', 'Portugal', 'US', 'Spain', 'France', 'Germany',
       'Argentina', 'Chile', 'Australia', 'Austria', 'South Africa',
       'New Zealand', 'Israel', 'Hungary', 'Greece', 'Romania', 'Mexico',
       'Canada', nan, 'Turkey', 'Czech Republic', 'Slovenia',
       'Luxembourg', 'Croatia', 'Georgia', 'Uruguay', 'England',
       'Lebanon', 'Serbia', 'Brazil', 'Moldova', 'Morocco', 'Peru',
       'India', 'Bulgaria', 'Cyprus', 'Armenia', 'Switzerland',
       'Bosnia and Herzegovina', 'Ukraine', 'Slovakia', 'Macedonia',
       'China', 'Egypt'], dtype=object)

In [4]:
df=df[df['country']=='US']

# EDA

In [5]:
df=df.drop_duplicates().reset_index(drop=True)
len(df)

50457

In [6]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
1,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
2,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
3,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
4,US,"Slightly reduced, this wine offers a chalky, t...",NaN,87,34.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini


In [7]:
df['designation'].fillna('unknown', inplace = True)
df['region_1'].fillna('unknown', inplace = True)
df['region_2'].fillna('unknown', inplace = True)
df['taster_name'].fillna('unknown', inplace = True)
df['taster_twitter_handle'].fillna('unknown', inplace = True)
df['price'].fillna((df['price'].median()), inplace = True)
pd.isna(df).sum()

country                  0
description              0
designation              0
points                   0
price                    0
province                 0
region_1                 0
region_2                 0
taster_name              0
taster_twitter_handle    0
title                    0
variety                  0
winery                   0
dtype: int64

In [8]:
print(df.columns)

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery'],
      dtype='object')


Checking Outliers for Price variable

In [9]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
#checking for extreme outliers outside 3 IQR range as these can be safely removed
outlierdf=(df[((df['price']< (Q1 - 3* IQR)) |(df['price'] > (Q3 + 3 * IQR)))])
outlierdf['price'].unique()

array([ 125.,  200.,  160.,  130.,  150.,  175.,  300.,  145.,  260.,
        195.,  155.,  210.,  230.,  135.,  190.,  140.,  165.,  250.,
        235.,  225.,  138.,  500.,  240.,  180.,  625.,  265.,  185.,
        275.,  179.,  325.,  215.,  450.,  750.,  170.,  129.,  219.,
        245.,  126.,  350.,  199.,  400.,  147.,  136.,  220., 2013.,
        290.,  335.,  149.,  168.])

In [10]:
df=df[~((df['price']< (Q1 - 3* IQR)) |(df['price'] > (Q3 + 3 * IQR)))]
len(df)

49687

# TASK 2

In [11]:
quantile = list(df.points.quantile([0, 0.25, 0.5, 0.75, 1.0]))
y_binned = []
for i in df['points']:
    if i<=quantile[1]:y_binned.append(1)
    elif i<=quantile[2]:y_binned.append(2)
    elif i<=quantile[3]:y_binned.append(3)
    else: y_binned.append(4)

In [12]:
from category_encoders import TargetEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
#preprocess = make_column_transformer((TargetEncoder(),['region_1','variety','winery']),(StandardScaler(), cnt),(OneHotEncoder(handle_unknown='ignore',sparse=False), ['province','region_2','taster_name','taster_twitter_handle']),remainder='passthrough')


# Only text features

Using pre trained embedding Word2vec

In [13]:
df_text=df[['description','title','points']]
df_text.head()

,description,title,points
0,"Tart and snappy, the flavors of lime flesh and...",Rainstorm 2013 Pinot Gris (Willamette Valley),87
1,"Pineapple rind, lemon pith and orange blossom ...",St. Julian 2013 Reserve Late Harvest Riesling ...,87
2,"Much like the regular bottling from 2012, this...",Sweet Cheeks 2012 Vintner's Reserve Wild Child...,87
3,"Soft, supple plum envelopes an oaky structure ...",Kirkland Signature 2011 Mountain Cuvée Caberne...,87
4,"Slightly reduced, this wine offers a chalky, t...",Louis M. Martini 2012 Cabernet Sauvignon (Alex...,87


In [14]:
quantile = list(df_text.points.quantile([0, 0.25, 0.5, 0.75, 1.0]))
y_binned = []
for i in df_text['points']:
    if i<=quantile[1]:y_binned.append(1)
    elif i<=quantile[2]:y_binned.append(2)
    elif i<=quantile[3]:y_binned.append(3)
    else: y_binned.append(4)

In [28]:
from sklearn.model_selection import train_test_split
X = df_text.drop(['points'],axis=1)
y=df_text['points']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,stratify=y_binned)

In [29]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [17]:
import spacy

creating word embeddings of Description and Title Columns

In [18]:
#load large spacy model
nlp = spacy.load("en_core_web_lg", disable=["tagger", "parser", "ner"])

#vectorize text columns
description_vectors = [nlp(row).vector for row in X_train.description]
title_vectors = [nlp(row).vector for row in X_train.title]


description_vectors_column = np.array(description_vectors)
title_vectors_column = np.array(title_vectors)

#add vector columns to a new dataframe
X_train1 = np.concatenate((description_vectors_column, title_vectors_column), axis=1)

In [19]:
X_train1=pd.DataFrame(X_train1)
X_train1.head()

,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599
0,-0.049963,0.130516,-0.035949,-0.103420,0.102252,0.030422,-0.027538,-0.005103,-0.069120,1.402075,...,0.042131,0.234346,-0.059009,-0.005550,-0.069646,-0.136891,0.021059,-0.246376,0.166911,0.014470
1,-0.154634,0.156198,-0.100224,-0.039200,0.079582,0.155891,-0.045321,0.003744,-0.044231,1.605355,...,-0.086565,0.207732,-0.133429,-0.088761,-0.353320,-0.132486,-0.121037,-0.285210,-0.070915,-0.171054
2,-0.042887,0.143630,-0.147088,-0.194524,0.195357,0.093230,0.016135,0.067777,0.012960,1.551056,...,-0.009680,0.171223,-0.069987,0.008739,-0.058367,-0.090819,0.088398,-0.192220,0.129839,-0.051605
3,-0.030193,0.180387,-0.113274,-0.164316,0.047304,0.011936,0.024298,-0.096466,-0.055957,1.593055,...,0.060139,0.054246,0.029309,-0.003235,-0.099709,-0.088300,0.027779,-0.214474,0.043602,0.022546
4,-0.039621,0.175381,-0.088664,-0.072200,0.062930,0.114613,-0.109622,-0.017670,-0.011668,1.567555,...,-0.032888,0.220764,-0.184462,0.142146,-0.235109,-0.221714,0.013727,-0.308778,0.064067,-0.212368


In [20]:
r1=linear_model.Ridge().fit(X_train1,y_train)
r1.score(X_train1,y_train)

0.6228999386746772

In [21]:
#vectorize text columns
description_vectors = [nlp(row).vector for row in X_test.description]
title_vectors = [nlp(row).vector for row in X_test.title]

#stack the vectors vertically
description_vectors_column = np.array(description_vectors)
title_vectors_column = np.array(title_vectors)

#add vector columns to a new dataframe
X_test1 = np.concatenate((description_vectors_column, title_vectors_column), axis=1)

In [22]:
r1.score(X_test1,y_test)

0.6090365601234655

This doesn't improve when compared to Task1 1.3(bag of words and improvement) which only dealt with text features.

Adding BOW model to this 

In [23]:
X_train2=X_train[['description','title']]
X_train2.head()

,description,title
0,"Pale yellow-pink in color, this sparkler shows...",Riverbench 2015 Cork Jumper Brut Rosé Sparklin...
1,"Spicy, tangy and tart, this well-made wine is ...",Quivira 2015 Zinfandel (Dry Creek Valley)
2,"This wine is brooding, with aromas of dark che...",Hedges 2013 Descendants Liégeois Dupont Les Go...
3,"This wine offers notes of smoke, tire rubber a...",Wilridge 2012 Rachel Red (Washington)
4,You can taste the California sun in every sip ...,Robinson Family Vineyards 2010 Estate Cabernet...


In [24]:
X_train3=pd.concat([X_train1,X_train2],axis=1)
X_train3.head()

,0,1,2,3,4,5,6,7,8,9,...,592,593,594,595,596,597,598,599,description,title
0,-0.049963,0.130516,-0.035949,-0.103420,0.102252,0.030422,-0.027538,-0.005103,-0.069120,1.402075,...,-0.059009,-0.005550,-0.069646,-0.136891,0.021059,-0.246376,0.166911,0.014470,"Pale yellow-pink in color, this sparkler shows...",Riverbench 2015 Cork Jumper Brut Rosé Sparklin...
1,-0.154634,0.156198,-0.100224,-0.039200,0.079582,0.155891,-0.045321,0.003744,-0.044231,1.605355,...,-0.133429,-0.088761,-0.353320,-0.132486,-0.121037,-0.285210,-0.070915,-0.171054,"Spicy, tangy and tart, this well-made wine is ...",Quivira 2015 Zinfandel (Dry Creek Valley)
2,-0.042887,0.143630,-0.147088,-0.194524,0.195357,0.093230,0.016135,0.067777,0.012960,1.551056,...,-0.069987,0.008739,-0.058367,-0.090819,0.088398,-0.192220,0.129839,-0.051605,"This wine is brooding, with aromas of dark che...",Hedges 2013 Descendants Liégeois Dupont Les Go...
3,-0.030193,0.180387,-0.113274,-0.164316,0.047304,0.011936,0.024298,-0.096466,-0.055957,1.593055,...,0.029309,-0.003235,-0.099709,-0.088300,0.027779,-0.214474,0.043602,0.022546,"This wine offers notes of smoke, tire rubber a...",Wilridge 2012 Rachel Red (Washington)
4,-0.039621,0.175381,-0.088664,-0.072200,0.062930,0.114613,-0.109622,-0.017670,-0.011668,1.567555,...,-0.184462,0.142146,-0.235109,-0.221714,0.013727,-0.308778,0.064067,-0.212368,You can taste the California sun in every sip ...,Robinson Family Vineyards 2010 Estate Cabernet...


In [25]:
import nltk
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    porter_stemmer=nltk.PorterStemmer()
    words = [porter_stemmer.stem(word) for word in words]
    return words


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
count_vect = TfidfVectorizer(tokenizer=Tokenizer)
preprocess3=make_column_transformer((count_vect,'description'),(count_vect,'title'),remainder='passthrough')
preprocess4=make_pipeline(preprocess3,linear_model.Ridge())
preprocess4.fit(X_train3,y_train)
preprocess4.score(X_train3,y_train)

0.8707159567162902

In [30]:
#vectorize text columns
description_vectors = [nlp(row).vector for row in X_test.description]
title_vectors = [nlp(row).vector for row in X_test.title]

#stack the vectors vertically
description_vectors_column = np.array(description_vectors)
title_vectors_column = np.array(title_vectors)

#add vector columns to a new dataframe
X_test1 = np.concatenate((description_vectors_column, title_vectors_column), axis=1)
X_test1=pd.DataFrame(X_test1)
X_test2=X_test[['description','title']]
X_test3=pd.concat([X_test1,X_test2],axis=1)
X_test3.head()

,0,1,2,3,4,5,6,7,8,9,...,592,593,594,595,596,597,598,599,description,title
0,-0.132519,0.164563,-0.099505,-0.033780,0.166579,0.121821,0.077894,-0.045162,-0.048361,1.478627,...,-0.212203,0.037839,-0.130836,-0.167290,-0.133314,-0.324056,0.161631,0.079690,Dry Gewürztraminer like this makes a great pai...,Husch 2013 Estate Bottled Gewurztraminer (Ande...
1,-0.092554,0.156035,-0.124652,-0.135731,0.068801,0.065038,-0.034220,-0.031794,-0.122094,1.905666,...,-0.068657,0.274172,-0.185290,-0.274044,0.058203,-0.192652,0.013831,0.011219,The grapes don't have any sweetness or concent...,Merry Cellars 2006 Syrah (Washington)
2,-0.102226,0.229831,-0.058515,-0.180138,0.038241,0.148641,-0.093580,0.017693,-0.024046,1.561335,...,0.015462,0.181671,-0.063941,-0.363467,0.219172,-0.285667,0.058956,-0.169007,"Oodles of rich, ripe fruit in this soft, gentl...",Leal Vineyards 2005 MacWilliamson Vineyard Pet...
3,-0.080322,0.194454,-0.066917,-0.102330,0.119923,0.084786,-0.026129,-0.014593,-0.026104,1.774154,...,-0.170423,-0.052667,-0.097661,-0.224405,-0.049813,-0.185099,0.049222,-0.230130,Young vine flavors of bright raspberry fruit s...,Season 2012 Malbec (Southern Oregon)
4,-0.011857,0.234463,-0.159766,-0.139863,0.119356,0.065522,-0.029372,0.038752,-0.091792,1.673595,...,-0.105803,0.076078,0.010753,-0.179679,0.082713,-0.167223,-0.052095,-0.055761,Earthy aromas of juniper and forest floor give...,Sanford 2014 Sanford & Benedict Vineyard Pinot...


In [31]:
preprocess4.score(X_test3,y_test)

0.7563844806124024

Considering BOW and Word2Vec embedding for text features only, there is a slight improvement from 0.743(Task 1.3) to 0.756.

# Considering all features (text and non text)

In [32]:
quantile = list(df.points.quantile([0, 0.25, 0.5, 0.75, 1.0]))
y_binned = []
for i in df['points']:
    if i<=quantile[1]:y_binned.append(1)
    elif i<=quantile[2]:y_binned.append(2)
    elif i<=quantile[3]:y_binned.append(3)
    else: y_binned.append(4)

In [33]:
from sklearn.model_selection import train_test_split
X = df.drop(['points','country','designation'],axis=1)
y=df['points']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,stratify=y_binned)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [34]:
#load large spacy model
nlp = spacy.load("en_core_web_lg", disable=["tagger", "parser", "ner"])

#vectorize text columns
description_vectors = [nlp(row).vector for row in X_train.description]
title_vectors = [nlp(row).vector for row in X_train.title]


description_vectors_column = np.array(description_vectors)
title_vectors_column = np.array(title_vectors)

#add vector columns to a new dataframe
X_train1 = np.concatenate((description_vectors_column, title_vectors_column), axis=1)

In [35]:
X_train1=pd.DataFrame(X_train1)
X_train2=X_train
X_train3=pd.concat([X_train1,X_train2],axis=1)
X_train3.head()

,0,1,2,3,4,5,6,7,8,9,...,description,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,-0.049963,0.130516,-0.035949,-0.103420,0.102252,0.030422,-0.027538,-0.005103,-0.069120,1.402075,...,"Pale yellow-pink in color, this sparkler shows...",44.0,California,Santa Maria Valley,Central Coast,Matt Kettmann,@mattkettmann,Riverbench 2015 Cork Jumper Brut Rosé Sparklin...,Sparkling Blend,Riverbench
1,-0.154634,0.156198,-0.100224,-0.039200,0.079582,0.155891,-0.045321,0.003744,-0.044231,1.605355,...,"Spicy, tangy and tart, this well-made wine is ...",25.0,California,Dry Creek Valley,Sonoma,Virginie Boone,@vboone,Quivira 2015 Zinfandel (Dry Creek Valley),Zinfandel,Quivira
2,-0.042887,0.143630,-0.147088,-0.194524,0.195357,0.093230,0.016135,0.067777,0.012960,1.551056,...,"This wine is brooding, with aromas of dark che...",30.0,Washington,Red Mountain,Columbia Valley,Sean P. Sullivan,@wawinereport,Hedges 2013 Descendants Liégeois Dupont Les Go...,Syrah,Hedges
3,-0.030193,0.180387,-0.113274,-0.164316,0.047304,0.011936,0.024298,-0.096466,-0.055957,1.593055,...,"This wine offers notes of smoke, tire rubber a...",20.0,Washington,Washington,Washington Other,Sean P. Sullivan,@wawinereport,Wilridge 2012 Rachel Red (Washington),Red Blend,Wilridge
4,-0.039621,0.175381,-0.088664,-0.072200,0.062930,0.114613,-0.109622,-0.017670,-0.011668,1.567555,...,You can taste the California sun in every sip ...,65.0,California,Stags Leap District,Napa,unknown,unknown,Robinson Family Vineyards 2010 Estate Cabernet...,Cabernet Franc,Robinson Family Vineyards


In [36]:
cnt=list(X.columns[X.dtypes !=object])
cnt

['price']

In [37]:
count_vect = TfidfVectorizer(tokenizer=Tokenizer)
preprocess = make_column_transformer((TargetEncoder(),['region_1','variety','winery']),(StandardScaler(), cnt),(OneHotEncoder(handle_unknown='ignore',sparse=False), ['province','region_2','taster_name','taster_twitter_handle']),(count_vect,'description'),(count_vect,'title'),remainder="passthrough")
preprocess4=make_pipeline(preprocess,linear_model.Ridge())
preprocess4.fit(X_train3,y_train)
preprocess4.score(X_train3,y_train)

0.8761292553446403

In [38]:
#vectorize text columns
description_vectors = [nlp(row).vector for row in X_test.description]
title_vectors = [nlp(row).vector for row in X_test.title]

#stack the vectors vertically
description_vectors_column = np.array(description_vectors)
title_vectors_column = np.array(title_vectors)

#add vector columns to a new dataframe
X_test1 = np.concatenate((description_vectors_column, title_vectors_column), axis=1)
X_test1=pd.DataFrame(X_test1)
X_test2=X_test
X_test3=pd.concat([X_test1,X_test2],axis=1)
X_test3.head()

,0,1,2,3,4,5,6,7,8,9,...,description,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,-0.132519,0.164563,-0.099505,-0.033780,0.166579,0.121821,0.077894,-0.045162,-0.048361,1.478627,...,Dry Gewürztraminer like this makes a great pai...,14.0,California,Anderson Valley,unknown,Jim Gordon,@gordone_cellars,Husch 2013 Estate Bottled Gewurztraminer (Ande...,Gewürztraminer,Husch
1,-0.092554,0.156035,-0.124652,-0.135731,0.068801,0.065038,-0.034220,-0.031794,-0.122094,1.905666,...,The grapes don't have any sweetness or concent...,32.0,Washington,Washington,Washington Other,Paul Gregutt,@paulgwine,Merry Cellars 2006 Syrah (Washington),Syrah,Merry Cellars
2,-0.102226,0.229831,-0.058515,-0.180138,0.038241,0.148641,-0.093580,0.017693,-0.024046,1.561335,...,"Oodles of rich, ripe fruit in this soft, gentl...",28.0,California,San Benito County,Central Coast,unknown,unknown,Leal Vineyards 2005 MacWilliamson Vineyard Pet...,Petite Sirah,Leal Vineyards
3,-0.080322,0.194454,-0.066917,-0.102330,0.119923,0.084786,-0.026129,-0.014593,-0.026104,1.774154,...,Young vine flavors of bright raspberry fruit s...,30.0,Oregon,Southern Oregon,Southern Oregon,Paul Gregutt,@paulgwine,Season 2012 Malbec (Southern Oregon),Malbec,Season
4,-0.011857,0.234463,-0.159766,-0.139863,0.119356,0.065522,-0.029372,0.038752,-0.091792,1.673595,...,Earthy aromas of juniper and forest floor give...,70.0,California,Sta. Rita Hills,Central Coast,Matt Kettmann,@mattkettmann,Sanford 2014 Sanford & Benedict Vineyard Pinot...,Pinot Noir,Sanford


In [39]:
preprocess4.score(X_test3,y_test)

0.7728765105104387

Considering BOW and Word2Vec embedding for all features combined, there is a slight improvement from 0.76(Task 1.4 without word2Vec) to 0.772.